### Structured Streaming

Spark proporciona dous mecanismos para traballar con fluxos de datos (streams):
- Spark Streaming
- Structured Streaming

**Spark Streaming**

Spark Streaming é unha libraría separada en Spark para procesar fluxos de datos. Proporciona a API DStream e traballa directamente sobre RDD. (máis orientado a Batch processing)

**Structured Streaming**

Funciona sobre Spark SQL, polo que traballa con DataSets/Dataframes. (máis orientado a real-time processing)

Nota: Todo parece indicar que Structured Streaming se convertirá no 'estándar' ao tratarse dunha ferramenta de máis alto nivel, se ben Spark Streaming é mais flexible e ao traballar directamente con RDDs permite obter mellor rendemento

#### Spark App que conta palabras a través dun fluxo de datos vía Socket TCP

In [ ]:
# Exemplo: aplicación que lee e conta palabras que recibe por un socket TCP.
# https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#overview

In [ ]:
# Resumo da estrutura da App
# - Importamos librarías necesarias
# - Creamos SparkSession
# - Creamos Dataframe do Stream de entrada
# - Lanzamos a consulta que abre a lectura do fluxo e escribe resultadoss

In [ ]:
# import dalgunhas funcións que usaremos para o tramento de texto
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [ ]:
# Esta cela é opcional, xa que Jupyter arranca automaticamente unha SparkSession ao lanzar o notebook

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate() 

In [ ]:
# lines == Dataframe que representa o fluxo de entrada de datos a partir da conexión ao socket TCP
# Definimos o socket en localhost, porto 9999
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Divide as liñas en palabras
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Xenera a conta de palabras
wordCounts = words.groupBy("word").count()

In [ ]:
# O DataFrame lines contén unicamente unha columna 'value' co texto de cada liña que recibe do socket
lines.printSchema()

In [ ]:
# Abre unha terminal a lanza netcat sobre o porto 9999
# Utilizarás a terminal como fonte de entrada de datos
# Os datos que introduzas pola terminal serán os que lea a Spark Application
# nc -lk 9999

In [ ]:
# Lanza a consulta que imprime a conta de palabras por consola

query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

# Antes de lanzar a query é importante abrir un socket nunha terminal co seguinte código:
# nc -lk 9999
query.awaitTermination()

In [ ]:
# Parece que non hai maneira de parar o query.awaitTermination()
# Unha solución é parar o notebook/kernel
query.stop()